# --- Zillow project: Home value prediction --- 

# Preprocessing

In [1]:
import pandas as pd

In long run, we need to develop a reuseble processes of data transformation

Data preprocessing involves many steps which might cause laborious effort to fix issue or change. Thus, we can give a preprocessing tool instead of static scripts or mannully preprocesse data file. We will be able to change things quickly and effectively while we building our models. 

The process would include several properties:
    1. convert raw data to modeling format datasets.
    2. static 1:1 mapping of inputs (raw data) to output(model data)
    3. Transformation should not depened on the size of datasets
    4. records should remain unchanged or not dropped 
    
Here is what we will do with some feature engineering and missing value imputation:
    1. Filtering features (drop unnecessary columns)
    2. encoding missing values with reasonable values (each feature variable)
    3. one hot encoding for categorical variables for ML modeling

In [2]:
# import functions
def Load_data(dataset, verbose=False):
    '''load one of Zillow dataset'''
    df = pd.read_csv('raw_data/{0}.csv'.format(dataset))
    
    if verbose:
        print('\n{0:*^80}'.format('In {0} dataset'.format(dataset)))
        print('\nit has {0} rows and {1} columns'.format(*df.shape))
        print('\n{0:*^80}'.format(' The columns are'))
        print(df.columns)
        print('\n{0:*^80}'.format(' The top 5 rows are'))
        print(df.head())
    
    return df

def select_per_parcel(df):
    '''drop duplicated to only record for each unique parcel'''
    
    count_per_parcel = df.groupby('parcelid').size()
    more_than_one = df[df.parcelid.isin(count_per_parcel[count_per_parcel > 1].index)]
    only_one = df[df.parcelid.isin(count_per_parcel[count_per_parcel == 1].index)]
    selected_more_than_one = more_than_one.sort_values(by='transactiondate', ascending=False).groupby('parcelid').head(1)
    All_unique_df = pd.concat([only_one, selected_more_than_one])
    
    print(All_unique_df.shape)
    return All_unique_df

def get_data(dataset):
    '''create training dataset 2016 or the test dataset 2017'''
    
    year = {'train': 2016, 'test': 2017}[dataset]
    train = Load_data('train_{0}'.format(year))
    properties = Load_data('properties_{0}'.format(year))
    merge_data = pd.merge(train, properties, how='left', on='parcelid')
    
    if dataset =='train':
        merge_data = select_per_parcel(merge_data)
        
    new_merge = merge_data.pop('logerror')
    return merge_data, new_merge

def mean_absolute_error(new_merge_real, new_merge_predict):
    return np.mean(np.abs(new_merge_real - new_merge_predict))   

--- Create object-oriented Programming (OOP)---

OOP is very useful to allow us keep information derived from the training dataset and the code to perform the preprocessing in same place

We'll create a preprocessor object (class) from the training data that can be applied to the test data as well as any new data in production. We'll use the fit and transform paradigm to ensure we're only learning inforamtion from the training data (and not the test data).

In [3]:
class preprocessor:
    
    def __init__(self, cols_to_filter=None,datecols=None):
        
        self.cols_to_filter = cols_to_filter
        self.datecols = datecols
    
    def fit(self, X, y=None):
        """learn any information from the training data we may need to transform the test data"""
        
        self.was_fit = True
        # filter
        X_new = X.drop(self.cols_to_filter, axis=1)
        
        categorical_features = X_new.dtypes[X_new.dtypes == 'object'].index
        self.categorical_features = [x for x in categorical_features if 'date' not in x]
        
        one_hot_encoding = pd.get_dummies(X_new, columns=self.categorical_features, dummy_na=True)
        self.colnames = one_hot_encoding.columns
        del one_hot_encoding
        return self
    
    def transform(self, X, y=None):
        """transform the training or test data learned from fit"""
        if not self.was_fit:
            raise Error('need tto fit preprocessor first')
        
        X_new = X.drop(self.cols_to_filter, axis=1)
        
        # fill null value
        X_new = X_new.fillna(0)
        
        X_new =pd.get_dummies(X_new, columns=self.categorical_features, dummy_na=True)
        newcols = set(self.colnames) - set(X_new.columns)
        for i in newcols:
            X_new[i] = 0
        
        X_new = X_new[self.colnames]
        
        # convert transactiondate to appropriate data type in columns
        if self.datecols:
            for col in self.datecols:
                X_new[col + '_month'] = pd.to_datetime(X_new[col]).apply(lambda x: x.month)
                X_new[col + '_year'] = pd.to_datetime(X_new[col]).apply(lambda x: x.year)
                X_new = X_new.drop(col, axis=1)      
        return X_new
    
    def fit_transform(self, X, y=None):
        '''fit and transform wrapper method for sklearn pipeline purpose use'''
        return self.fit(X).transform(X)

In [4]:
train_X, train_y = get_data(dataset='train')

E:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2961: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(90150, 60)


In [5]:
train_X.head()

,parcelid,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,2016-01-01,1.0,NaN,NaN,2.0,3.0,NaN,4.0,2.0,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13
1,14366692,2016-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN
2,12098116,2016-01-01,1.0,NaN,NaN,3.0,2.0,NaN,4.0,3.0,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13
3,12643413,2016-01-02,1.0,NaN,NaN,2.0,2.0,NaN,4.0,2.0,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,NaN,6.037296e+13
4,14432541,2016-01-02,NaN,NaN,NaN,2.5,4.0,NaN,NaN,2.5,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13


In [6]:
# ---Checking for null values---
train_X.loc[:, train_X.isna().sum() > 0].min()

airconditioningtypeid                    1
architecturalstyletypeid                 2
basementsqft                           100
buildingclasstypeid                      4
buildingqualitytypeid                    1
calculatedbathnbr                        1
decktypeid                              66
finishedfloor1squarefeet                44
calculatedfinishedsquarefeet             2
finishedsquarefeet12                     2
finishedsquarefeet13                  1056
finishedsquarefeet15                   560
finishedsquarefeet50                    44
finishedsquarefeet6                    257
fireplacecnt                             1
fullbathcnt                              1
garagecarcnt                             0
garagetotalsqft                          0
hashottuborspa                        True
heatingorsystemtypeid                    1
lotsizesquarefeet                      167
poolcnt                                  1
poolsizesum                             28
pooltypeid1

In [7]:
p = preprocessor(cols_to_filter=['rawcensustractandblock', 'censustractandblock', 'propertyzoningdesc', 
                                 'regionidneighborhood', 'regionidzip', 'parcelid'], datecols=['transactiondate'])
p.fit(train_X)

In [8]:
train_X_transformed = p.transform(train_X)

In [9]:
train_X_transformed.head()

,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,...,propertycountylandusecode_73,propertycountylandusecode_8800,propertycountylandusecode_96,propertycountylandusecode_nan,fireplaceflag_True,fireplaceflag_nan,taxdelinquencyflag_Y,taxdelinquencyflag_nan,transactiondate_month,transactiondate_year
0,1.0,0.0,0.0,2.0,3.0,0.0,4.0,2.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,2016
1,0.0,0.0,0.0,3.5,4.0,0.0,0.0,3.5,0.0,0.0,...,0,0,0,0,0,0,0,0,1,2016
2,1.0,0.0,0.0,3.0,2.0,0.0,4.0,3.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,2016
3,1.0,0.0,0.0,2.0,2.0,0.0,4.0,2.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,2016
4,0.0,0.0,0.0,2.5,4.0,0.0,0.0,2.5,0.0,0.0,...,0,0,0,0,0,0,0,0,1,2016


In [10]:
assert all(train_X_transformed.isna().sum() == 0) 